In [1]:
%pylab inline
import numpy as np
import pandas
import os
import re
import sklearn
import calendar
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [14]:
np.arange(0,60*24,10)/10

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143])

In [2]:
ff=lambda k : np.floor(((k.hour*60)+k.minute)/10.0)
correcttime= lambda k : k.time()
convert_to_string=lambda k : '-'+str(int(k))
datetimeslot=lambda k : str(k.date())+'-'+str(int(np.floor(((k.hour*60)+k.minute)/10.0)))
returnweekday = lambda k : calendar.weekday(int(k[:4]),int(k[5:7]),int(k[8:10]))
returntimeslot= lambda k : k[11:]
returndate=lambda k : k[:10]
returnday=lambda k : k[8:10]
getdate = lambda k : str(k.date())
in_int= lambda k : int(k[11:])
typecast_int = lambda k : int(k)
typecast_str = lambda k : str(k)

### Reading all Directories

In [3]:
os.chdir('./training_data')
dirlist=['weather_data','cluster_map','order_data','poi_data']
filesList=[]
for dir_ in dirlist:
    unique=np.unique(os.listdir(dir_))
    filesList.append(unique[unique.shape[0]/2:])

## Weather Data

In [21]:
def loadWeather(filesList):
    weatherlist=filesList[0]
    unique,counts=np.unique(weatherlist,return_counts=True)
    weatherlist=unique
    weather=pandas.read_table('./weather_data/'+weatherlist[0],names=['date','time','weather','temperature','pm25'],delim_whitespace=True,parse_dates=True, infer_datetime_format=True, keep_date_col=True)
    counter=1
    while counter<len(weatherlist):
        weather=weather.append(pandas.read_table('./weather_data/'+weatherlist[counter],names=['date','time','weather','temperature','pm25'],delim_whitespace=True,parse_dates=True, infer_datetime_format=True, keep_date_col=True),ignore_index=True,)
        counter+=1
    weather=weather.sort_values(['date','time'])
    weather.time=pandas.to_datetime(weather.time).apply(correcttime)
    weather.time=weather.time.apply(ff).apply(np.floor)
    uniqueDates=weather.date.copy()
    weather.date=weather.date+weather.time.apply(convert_to_string)
    weather.drop('time',axis=1,inplace=True)
    print weather.shape
    return weather,uniqueDates


In [4]:
def mean_weather(weather,mean=True):
    w=weather.groupby(by=['date'])
    if mean:
        meanedWeather=w.mean()
        meanedWeather.weather.apply(np.ceil)
    else:
        mean_weather=w.mode()
    return meanedWeather
    
# meanedWeather.join(np.arange(1,145,1),on='time',how='right')

In [5]:
def fill_empty(dates):
    p=pandas.DataFrame(([dates[0]]*144),columns=['date'])
    p.date=p.date+'-'+map(str,arange(0,144,1))
    counter=1
    while counter<dates.shape[0]:
        new=pandas.DataFrame(([dates[counter]]*144),columns=['date'])
        new.date=new.date+pandas.Series(np.arange(0,144,1)).apply(convert_to_string)
        p=p.append(new,ignore_index=True)
        counter+=1
    return p


In [6]:
def fillup_Allvalues(ww):
    vals=np.array(ww.values[:,1:],dtype=np.float32)
    while True:
        nans=np.where(np.isnan(vals))[0]
        if len(nans)==0:
            break
        indexes=np.unique(nans)
        for i in indexes:
            firstlist=np.zeros((0,3))
            secondlist=np.zeros((0,3))
            j=i-3
            while j<i:
                if np.isnan(vals[j])[0] ==False:
                    if firstlist.shape[0]==0:
                        firstlist=vals[j]
                        firstlist.reshape((-1,3))
                    else:
                        firstlist=np.vstack((firstlist,vals[j]))

                j+=1
            j=i+3
            while j<vals.shape[0] and j>i:
                if np.isnan(vals[j])[0] ==False:
                    if secondlist.shape[0]==0:
                        secondlist=vals[j]
                        secondlist.reshape((-1,3))
                    else:
                        secondlist=np.vstack((secondlist,vals[j]))
                j-=1
            new=None    
            if len(firstlist) > 0 and len(secondlist)>0:
                new=np.vstack((firstlist,secondlist))
            elif len(firstlist)>0:
                new=np.array(firstlist)
            elif len(secondlist)>0:
                new=np.array(secondlist)
            if new !=None:
    #             print new
    #             print stats.mode(new,axis=0)
    #             vals[i]=stats.mode(new,axis=0)[0]
                vals[i]=np.mean(new,axis=0)
                vals[i][0]=np.ceil(vals[i][0])
    ww.reset_index()
    ww.drop(['weather','temperature','pm25'],axis=1,inplace=True)
    temp=pandas.DataFrame(data=vals,columns=['weather','temperature','pm25'])
    ww[temp.columns]=temp
    return ww
# ww['weather'],ww['temperature'],ww['pm25']=vals


In [7]:
def get_weather(filesList):
    weather,uniqueDates=loadWeather(filesList)
    meanedWeather=mean_weather(weather)
    ww=fill_empty(np.unique(uniqueDates.values)).join(meanedWeather,on='date',how='left')
    meanedWeather=fillup_Allvalues(ww)
    meanedWeather=meanedWeather.set_index(['date'])
    return meanedWeather

In [9]:
meanedWeather=get_weather(filesList)
meanedWeather.weather=meanedWeather.weather.apply(np.ceil)

/home/rameez/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/rameez/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [ ]:
temp=meanedWeather.temperature.values
pm=meanedWeather.pm25
wea=meanedWeather.weather
length=arange(1,temp.shape[0]+1,1)

In [ ]:
plt.scatter(length,temp,color='b')

In [ ]:
plt.scatter(length,pm,color='b')

In [ ]:
plt.scatter(length,wea,color='b')

# Cluster Map

In [3]:
def load_cluster_map(dirlist,filesList):
    print "./"+dirlist[1]+'/'+filesList[1][0]
    cluster_map=pandas.read_table("./"+dirlist[1]+'/'+filesList[1][0],index_col=0,names=['region_id'])
    return cluster_map

In [5]:
cluster_map=load_cluster_map(dirlist,filesList)
cluster_map

./cluster_map/cluster_map


,region_id
90c5a34f06ac86aee0fd70e2adce7d8a,1
f2c8c4bb99e6377d21de71275afd6cd2,2
58c7a4888306d8ff3a641d1c0feccbe3,3
b26a240205c852804ff8758628c0a86a,4
4b9e4cf2fbdc8281b8a1f9f12b80ce4d,5
1cbfbdd079ef93e74405c53fcfff8567,6
929ec6c160e6f52c20a4217c7978f681,7
82cc4851f9e4faa4e54309f8bb73fd7c,8
b702e920dcd2765e624dc1ce3a770512,9
4f4041f7db0c7f69892d9b74c1a7efa1,10


# POI Data

In [4]:
def get_poi_matrix(filesList):
    print filesList[3][0]
    currentdata=[]
    with open('./'+dirlist[3]+'/'+filesList[3][0]) as f:
        currentdata=f.read()
    uniquelabels=np.unique(re.findall(r'\t(.*?):',currentdata))
#     print uniquelabels
    regionhashes=re.findall(r'(.*?)\t.*?\n',currentdata)
    uniquesize=uniquelabels.shape[0]
    lines=re.findall(r'.*?[\r\n]+',currentdata)
    poi_matrix=np.zeros((len(lines),uniquesize))

    for i,l in enumerate(lines):
        codes=np.array(re.findall(r'\t(.*?):',l))
        numbers=np.array(re.findall(r':([0-9]+)',l),dtype=int)
        poi_matrix[i][np.where((uniquelabels==codes.reshape((-1,1))))[1]]=numbers
#         if i==50:
#             print codes,len(codes)
#             print numbers, len(numbers)
#     print poi_matrix.shape
    return poi_matrix,uniquelabels,regionhashes

# uniquelabels.sort()

In [5]:
def compress(poimatrix):
    summed=np.sum(poimatrix,axis=0)
    poimatrix/=summed
    poimatrix=np.sum(poimatrix,axis=1)
    return poimatrix

In [6]:
def get_poi_dataframe(filesList,cluster_map,compress_it=False):
    poi_matrix,uniquelabels,regionhashes=get_poi_matrix(filesList)
    if compress_it:
        poi_matrix=compress(poi_matrix)
        poi_dataframe=pandas.DataFrame(data=poi_matrix,columns=['poi_info'])
    else:
        poi_dataframe=pandas.DataFrame(data=poi_matrix,columns=uniquelabels)
    poi_dataframe.insert(0,column='region_id',value=regionhashes)
    poi_dataframe.region_id=poi_dataframe.replace(cluster_map.index,cluster_map.region_id)
    poi_dataframe.set_index(keys='region_id',inplace=True)
    poi_dataframe.sort_index()
    return poi_dataframe

In [11]:
poi_dataframe=get_poi_dataframe(filesList,cluster_map,compress_it=False)
poi_dataframe.sort_index()
poi_dataframe.to_csv('poi_matrix',sep='\t')

poi_data


# Order Data

In [16]:
%%time
print filesList[2][0]
order_data=pandas.read_table('./'+dirlist[2]+'/'+filesList[2][0],names=['order_id','driver_id','passenger_id','start_region_hash','dest_region_hash','price','date','time'],infer_datetime_format=True,parse_dates=True,delim_whitespace=True,keep_date_col=True)
order_data.time=pandas.to_datetime(order_data.time,format='%H:%M:%S')
print order_data.shape

counter=1
while counter<len(filesList[2]):
    print filesList[2][counter]
    new=pandas.read_table('./'+dirlist[2]+'/'+filesList[2][counter],names=['order_id','driver_id','passenger_id','start_region_hash','dest_region_hash','price','date','time'],parse_dates=True,infer_datetime_format=True,delim_whitespace=True,keep_date_col=True)
    print new.shape
    new.time=pandas.to_datetime(new.time,format='%H:%M:%S')
    order_data=order_data.append(new,ignore_index=True)
    print order_data.shape
    counter+=1
order_data.time.apply(correcttime)
order_data.insert(7,'time_slots',order_data.time.apply(ff).apply(np.floor))
order_data.describe(include='all')

order_data_2016-01-01
(501287, 8)
order_data_2016-01-02
(322284, 8)
(823571, 8)
order_data_2016-01-03
(325179, 8)
(1148750, 8)
order_data_2016-01-04
(385818, 8)
(1534568, 8)
order_data_2016-01-05
(385623, 8)
(1920191, 8)
order_data_2016-01-06
(366661, 8)
(2286852, 8)
order_data_2016-01-07
(338085, 8)
(2624937, 8)
order_data_2016-01-08
(395983, 8)
(3020920, 8)
order_data_2016-01-09
(372408, 8)
(3393328, 8)
order_data_2016-01-10
(377213, 8)
(3770541, 8)
order_data_2016-01-11
(447198, 8)
(4217739, 8)
order_data_2016-01-12
(379198, 8)
(4596937, 8)
order_data_2016-01-13
(387710, 8)
(4984647, 8)
order_data_2016-01-14
(388371, 8)
(5373018, 8)
order_data_2016-01-15
(451292, 8)
(5824310, 8)
order_data_2016-01-16
(442293, 8)
(6266603, 8)
order_data_2016-01-17
(374411, 8)
(6641014, 8)
order_data_2016-01-18
(410210, 8)
(7051224, 8)
order_data_2016-01-19
(479227, 8)
(7530451, 8)
order_data_2016-01-20
(535823, 8)
(8066274, 8)
order_data_2016-01-21
(474340, 8)
(8540614, 8)
CPU times: user 3min 4s, sy

In [17]:
# temp.to_csv(string+str(i),sep='\t')# print order_data.sort([time])
# print order_data.start_region_hash
order_data.start_region_hash = order_data.start_region_hash.replace(cluster_map.index,cluster_map.region_id)

# timeslots=((((order_data.time.dt.hour*60)+order_data.time.dt.minute)+order_data.time.dt.second)/600)
# order_data.add(timeslots.apply(np.floor),axis='columns')

### Write to file

In [ ]:
order_data.to_csv('complete_data',sep='\t')

In [ ]:
# order_data.to_csv(['testing1','testing2','testing3'],sep='\t',chunksize=50000)
# order_data.to_csv('testing2',sep='\t',chunksize=50000)
order_data.shape[0]/21

### Read Created table from file

In [ ]:
order_data=pandas.read_table('complete_data',parse_dates=True, infer_datetime_format=True, keep_date_col=True,index_col=0)

In [ ]:
order_data

In [ ]:
grouped_Only_startHash=order_data.groupby([order_data.start_region_hash]).count()
grouped_OnlyLables=grouped_Only_startHash.order_id-grouped_Only_startHash.driver_id
grouped_OnlyLables.shape

In [18]:
order_data.date=order_data.date+order_data.time_slots.apply(convert_to_string)
order_data.drop(['time','time_slots','dest_region_hash','passenger_id'],axis=1,inplace=True)

### Getting Labels

In [19]:
func=lambda k : '-'+str(int(k))

In [20]:
def getlabels(order_data):
        v=order_data.groupby([order_data.start_region_hash,order_data.date])
        labels=pandas.DataFrame((v.order_id.count()-v.driver_id.count()),columns=['gap'])
        labels['number_of_orders']=v.order_id.count()
        labels.reset_index(inplace=True)
        return labels

In [21]:
labels=getlabels(order_data)

In [22]:
order_data=labels.copy()

In [23]:
# labels.drop(labels='number_of_orders',axis=1,inplace=True)
labels

,start_region_hash,date,gap,number_of_orders
0,1,2016-01-01-0,9,187
1,1,2016-01-01-1,7,198
2,1,2016-01-01-10,2,72
3,1,2016-01-01-100,5,279
4,1,2016-01-01-101,3,238
5,1,2016-01-01-102,9,283
6,1,2016-01-01-103,7,275
7,1,2016-01-01-104,7,280
8,1,2016-01-01-105,11,290
9,1,2016-01-01-106,8,293


## Writing labels

In [79]:
# print labels
labels.to_csv('labels',sep='\t')

## Reading Labels


In [25]:
labels=pandas.read_table('labels',index_col=0)
labels.gap.max()

3872

### PLOTTING

In [ ]:
plt.plot(arange(0,len(labels),1),labels.values)


In [ ]:
print grouped_OnlyLables

In [ ]:
start=1
while start<=66:
    plt.scatter(arange(1,len(grouped_OnlyLables.get((1)).values)+1,1),grouped_OnlyLables.get((1)).values)
    start+=1


## COMBINING DATA

In [24]:
order_data.drop('gap',axis=1,inplace=True)

In [25]:
order_data=order_data.join(poi_dataframe,on='start_region_hash',how='inner')
order_data=order_data.join(meanedWeather,on='date')
order_data

,start_region_hash,date,number_of_orders,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8,8#1,8#2,8#3,8#4,8#5,9#1#1,weather,temperature,pm25
0,1,2016-01-01-0,187,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,1.0,3.500000,177.000000
1,1,2016-01-01-1,198,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,1.0,3.000000,177.000000
2,1,2016-01-01-10,72,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,1.0,3.000000,206.000000
3,1,2016-01-01-100,279,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,12.000000,168.000000
4,1,2016-01-01-101,238,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,11.000000,154.000000
5,1,2016-01-01-102,283,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,11.000000,154.000000
6,1,2016-01-01-103,275,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,11.000000,154.000000
7,1,2016-01-01-104,280,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,10.000000,154.000000
8,1,2016-01-01-105,290,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,10.000000,154.000000
9,1,2016-01-01-106,293,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0,2.0,10.000000,146.000000


In [26]:
col=order_data.weather
order_data.drop('weather',axis=1,inplace=True)
order_data['weather']=col

In [27]:
order_data['week_day']=order_data.date.apply(returnweekday)

In [28]:
order_data['day']=order_data.date.apply(returnday)

In [29]:
order_data['date']=order_data.date.apply(returntimeslot)

In [30]:
order_data

,start_region_hash,date,number_of_orders,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8#2,8#3,8#4,8#5,9#1#1,temperature,pm25,weather,week_day,day
0,1,0,187,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,3.500000,177.000000,1.0,4,01
1,1,1,198,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,3.000000,177.000000,1.0,4,01
2,1,10,72,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,3.000000,206.000000,1.0,4,01
3,1,100,279,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,12.000000,168.000000,2.0,4,01
4,1,101,238,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,11.000000,154.000000,2.0,4,01
5,1,102,283,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,11.000000,154.000000,2.0,4,01
6,1,103,275,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,11.000000,154.000000,2.0,4,01
7,1,104,280,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,10.000000,154.000000,2.0,4,01
8,1,105,290,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,10.000000,154.000000,2.0,4,01
9,1,106,293,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,10.000000,146.000000,2.0,4,01


In [ ]:
# order_data.rename(index=str,columns={'order_id':'number_of_orders'},inplace=True)
# order_data

In [31]:
order_data.to_csv('final_data_complete',sep='\t')

## Load Variant

In [ ]:
order_data=pandas.read_table('final_data_variant',index_col=0)

## Loading Compressed Variant

In [ ]:
order_data=pandas.read_table('final_data_variant_compressed',index_col=0)

## Load Old

In [ ]:
order_data=pandas.read_table('final_data_',index_col=0)

# LOADING TEST DATA SET

In [7]:
# os.chdir('..')
os.chdir('./test_set')
dirlist=['weather_data','cluster_map','order_data','poi_data']
filesList=[]
for dir_ in dirlist:
    unique=np.unique(os.listdir(dir_))
    filesList.append(unique)
print filesList[2]

['test_order_data_2016-01-23' 'test_order_data_2016-01-25'
 'test_order_data_2016-01-27' 'test_order_data_2016-01-29'
 'test_order_data_2016-01-31']


In [22]:
meanedWeather=get_weather(filesList)
meanedWeather[:80]

/home/rameez/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


,weather,temperature,pm25
date,,,
2016-01-23-0,8.0,1.000000,69.000000
2016-01-23-1,8.0,1.000000,69.000000
2016-01-23-2,8.0,1.000000,69.000000
2016-01-23-3,8.0,1.000000,69.000000
2016-01-23-4,8.0,1.000000,69.000000
2016-01-23-5,8.0,1.000000,69.000000
2016-01-23-6,8.0,1.000000,69.000000
2016-01-23-7,8.0,1.000000,69.000000
2016-01-23-8,8.0,1.000000,69.000000


In [8]:

cluster_map=load_cluster_map(dirlist,filesList)
cluster_map

./cluster_map/cluster_map


,region_id
90c5a34f06ac86aee0fd70e2adce7d8a,1
f2c8c4bb99e6377d21de71275afd6cd2,2
58c7a4888306d8ff3a641d1c0feccbe3,3
b26a240205c852804ff8758628c0a86a,4
4b9e4cf2fbdc8281b8a1f9f12b80ce4d,5
1cbfbdd079ef93e74405c53fcfff8567,6
929ec6c160e6f52c20a4217c7978f681,7
82cc4851f9e4faa4e54309f8bb73fd7c,8
b702e920dcd2765e624dc1ce3a770512,9
4f4041f7db0c7f69892d9b74c1a7efa1,10


In [ ]:
cluster_map.sort_values()

In [9]:
poi_dataframe=get_poi_dataframe(filesList,cluster_map,compress_it=False)
poi_dataframe.sort_index()

poi_data


,1,1#1,1#10,1#11,1#2,1#3,1#4,1#5,1#6,1#7,...,7#1,7#2,7#3,8,8#1,8#2,8#3,8#4,8#5,9#1#1
region_id,,,,,,,,,,,,,,,,,,,,,
1.0,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,8051.0,83.0,0.0,...,0.0,0.0,0.0,166.0,0.0,7553.0,1743.0,2075.0,0.0,0.0
2.0,1245.0,83.0,0.0,2324.0,913.0,0.0,83.0,5644.0,166.0,0.0,...,0.0,0.0,0.0,996.0,166.0,3403.0,1826.0,2656.0,664.0,0.0
3.0,83.0,0.0,0.0,83.0,83.0,0.0,0.0,166.0,332.0,0.0,...,83.0,83.0,0.0,0.0,0.0,83.0,0.0,83.0,0.0,0.0
4.0,332.0,83.0,83.0,1162.0,415.0,0.0,166.0,2573.0,0.0,0.0,...,0.0,166.0,0.0,0.0,0.0,1577.0,249.0,1162.0,83.0,0.0
5.0,83.0,0.0,0.0,83.0,0.0,83.0,83.0,249.0,415.0,0.0,...,0.0,0.0,0.0,0.0,0.0,249.0,83.0,332.0,0.0,0.0
6.0,0.0,0.0,0.0,1162.0,498.0,0.0,83.0,664.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,913.0,166.0,83.0,0.0,0.0
7.0,3735.0,166.0,83.0,1577.0,1660.0,996.0,664.0,8715.0,0.0,332.0,...,0.0,0.0,0.0,1162.0,166.0,10624.0,14691.0,7470.0,83.0,0.0
8.0,6142.0,664.0,830.0,4980.0,1660.0,249.0,415.0,21082.0,83.0,166.0,...,0.0,0.0,0.0,996.0,498.0,13446.0,5893.0,9628.0,0.0,0.0
9.0,747.0,332.0,0.0,996.0,747.0,166.0,83.0,3486.0,249.0,249.0,...,0.0,0.0,0.0,166.0,0.0,1411.0,747.0,996.0,0.0,0.0


In [10]:
poi_dataframe.to_csv('poi_matrix',sep='\t')

In [20]:
poi_dataframe.sort_values('poi_info')

,poi_info
region_id,
45.0,0.106981
55.0,0.137201
62.0,0.159814
52.0,0.278540
36.0,0.364703
10.0,0.431584
40.0,0.433243
5.0,0.453589
32.0,0.486356


In [33]:
%%time
print filesList[2][0]
order_data=pandas.read_table('./'+dirlist[2]+'/'+filesList[2][0],names=['order_id','passenger_id','start_region_hash','dest_region_hash','date'],sep=',',infer_datetime_format=True,parse_dates=True,keep_date_col=True)
order_data.date=pandas.to_datetime(order_data.date)
print order_data.shape

counter=1
while counter<len(filesList[2]):
    print filesList[2][counter]
    new=pandas.read_table('./'+dirlist[2]+'/'+filesList[2][counter],names=['order_id','passenger_id','start_region_hash','dest_region_hash','date'],sep=',',parse_dates=True,infer_datetime_format=True,keep_date_col=True)
    print new.shape
    new.date=pandas.to_datetime(new.date)
    order_data=order_data.append(new,ignore_index=True)
    print order_data.shape
    counter+=1
# order_data.time.apply(correcttime)
# order_data.insert(6,'time_slots',order_data.time.apply(ff).apply(np.floor))
order_data

test_order_data_2016-01-23
(105891, 5)
test_order_data_2016-01-25
(109062, 5)
(214953, 5)
test_order_data_2016-01-27
(112054, 5)
(327007, 5)
test_order_data_2016-01-29
(125831, 5)
(452838, 5)
test_order_data_2016-01-31
(102285, 5)
(555123, 5)
CPU times: user 1.7 s, sys: 64 ms, total: 1.77 s
Wall time: 1.77 s


In [26]:
order_data.date=order_data.date.apply(datetimeslot)

In [27]:
func = lambda k: re.findall(r'_(.*)',k)[0]
returndistrict= lambda k : re.findall(r'(.*?)_.*',k)[0]

In [28]:
order_data.drop(['dest_region_hash','passenger_id'],axis=1,inplace=True)
order_data=order_data.groupby(['start_region_hash','date']).count()

In [29]:
def get_order_data(order_data,meanedWeather,poi_dataframe,cluster_map):
    order_data.start_region_hash.replace(to_replace=cluster_map.index,value=cluster_map.region_id,inplace=True)
#     order_data.date=order_data.date.apply(datetimeslot)
    
    order_data=order_data.join(poi_dataframe,on='start_region_hash',how='inner')
    order_data=order_data.join(meanedWeather,on='date')
    #     print sample.sort(['date','district_date_slot'])
    col=order_data.weather
    order_data.drop('weather',axis=1,inplace=True)
    order_data['weather']=col


#     order_data['week_day']=order_data.date.apply(returnweekday)
    return order_data

In [30]:
order_data.reset_index(inplace=True)

In [31]:
order_data=get_order_data(order_data,meanedWeather,poi_dataframe,cluster_map)
# order_data

In [32]:
order_data

,start_region_hash,date,order_id,poi_info,temperature,pm25,weather
0,50,2016-01-23-102,9,0.556785,-0.780463,47.042519,4.0
1,50,2016-01-23-103,8,0.556785,-2.000000,43.000000,3.0
2,50,2016-01-23-104,3,0.556785,-1.134855,45.871868,4.0
3,50,2016-01-23-114,2,0.556785,-1.613069,43.893211,4.0
4,50,2016-01-23-115,2,0.556785,-2.000000,42.000000,2.0
5,50,2016-01-23-116,1,0.556785,-1.724599,43.346073,4.0
6,50,2016-01-23-126,1,0.556785,-2.658210,46.117210,4.0
7,50,2016-01-23-127,1,0.556785,-4.000000,51.000000,1.0
8,50,2016-01-23-138,1,0.556785,-4.000000,56.000000,2.0
9,50,2016-01-23-139,1,0.556785,-4.000000,56.000000,2.0


In [23]:
# order_data['time_slot']=order_data.date.apply(returntimeslot)
order_data['week_day']=order_data.date.apply(returnweekday)
# order_data.date=order_data.date.apply(returndate)

In [28]:
order_data['day']=order_data.date.apply(returnday)

In [29]:
order_data['time_slot']=order_data.date.apply(returntimeslot)

In [30]:
order_data.time_slot=order_data.time_slot.apply(typecast_int)

In [31]:
order_data.date=order_data.date.apply(returndate)

In [32]:
order_data

,start_region_hash,date,order_id,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8#2,8#3,8#4,8#5,9#1#1,temperature,pm25,weather,day,time_slot
0,50,2016-01-23,9,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-0.780463,47.042519,4.0,23,102
1,50,2016-01-23,8,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-2.000000,43.000000,3.0,23,103
2,50,2016-01-23,3,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-1.134855,45.871868,4.0,23,104
3,50,2016-01-23,2,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-1.613069,43.893211,4.0,23,114
4,50,2016-01-23,2,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-2.000000,42.000000,2.0,23,115
5,50,2016-01-23,1,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-1.724599,43.346073,4.0,23,116
6,50,2016-01-23,1,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-2.658210,46.117210,4.0,23,126
7,50,2016-01-23,1,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-4.000000,51.000000,1.0,23,127
8,50,2016-01-23,1,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-4.000000,56.000000,2.0,23,138
9,50,2016-01-23,1,83.0,83.0,0.0,166.0,249.0,83.0,0.0,...,332.0,83.0,415.0,0.0,0.0,-4.000000,56.000000,2.0,23,139


In [35]:
g=order_data.sort(['date','time_slot','start_region_hash'])

/home/rameez/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [40]:
g.rename(g.columns={'order_id':'number_of_orders'})

SyntaxError: keyword can't be an expression (<ipython-input-40-4bee19da130b>, line 1)

In [37]:
g.reset_index(drop=True,inplace=True)

In [31]:
g.to_csv('./cleaned_full_test_without_orders',sep='\t')

In [38]:
g.to_csv('./cleaned_complete',sep='\t')

In [64]:
g.drop('order_id',axis=1,inplace=True)

In [65]:
g.to_csv('./cleaned_test',sep='\t')

In [39]:
g

,start_region_hash,date,order_id,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8#2,8#3,8#4,8#5,9#1#1,temperature,pm25,weather,day,time_slot
0,1,2016-01-23,64,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,7553.0,1743.0,2075.0,0.0,0.0,1.0,94.0,4.0,23,42
1,2,2016-01-23,5,1245.0,83.0,0.0,2324.0,913.0,0.0,83.0,...,3403.0,1826.0,2656.0,664.0,0.0,1.0,94.0,4.0,23,42
2,3,2016-01-23,2,83.0,0.0,0.0,83.0,83.0,0.0,0.0,...,83.0,0.0,83.0,0.0,0.0,1.0,94.0,4.0,23,42
3,4,2016-01-23,21,332.0,83.0,83.0,1162.0,415.0,0.0,166.0,...,1577.0,249.0,1162.0,83.0,0.0,1.0,94.0,4.0,23,42
4,5,2016-01-23,1,83.0,0.0,0.0,83.0,0.0,83.0,83.0,...,249.0,83.0,332.0,0.0,0.0,1.0,94.0,4.0,23,42
5,6,2016-01-23,6,0.0,0.0,0.0,1162.0,498.0,0.0,83.0,...,913.0,166.0,83.0,0.0,0.0,1.0,94.0,4.0,23,42
6,7,2016-01-23,30,3735.0,166.0,83.0,1577.0,1660.0,996.0,664.0,...,10624.0,14691.0,7470.0,83.0,0.0,1.0,94.0,4.0,23,42
7,8,2016-01-23,79,6142.0,664.0,830.0,4980.0,1660.0,249.0,415.0,...,13446.0,5893.0,9628.0,0.0,0.0,1.0,94.0,4.0,23,42
8,9,2016-01-23,4,747.0,332.0,0.0,996.0,747.0,166.0,83.0,...,1411.0,747.0,996.0,0.0,0.0,1.0,94.0,4.0,23,42
9,10,2016-01-23,1,0.0,0.0,0.0,0.0,83.0,83.0,0.0,...,166.0,0.0,166.0,0.0,0.0,1.0,94.0,4.0,23,42


In [119]:
# order_data.drop('index',axis=1,inplace=True)
order_data.columns

Index([u'start_region_hash', u'date', u'order_id', u'poi_info', u'temperature',
       u'pm25', u'weather', u'week_day', u'time_slot'],
      dtype='object')

In [68]:
grouped=order_data.groupby(['start_region_hash','date']).count()

# grouped=grouped.reset_index()
# grouped.date.unique().shape
# grouped.reset_index(inplace=True)
# plt.plot(grouped[grouped.date=='2016-01-29'].time_slot,grouped[grouped.date=='2016-01-29'].order_id)
grouped

order_id  poi_info  temperature  pm25  \
start_region_hash date                                                    
1                 2016-01-23-103         1         1            1     1   
                  2016-01-23-104         1         1            1     1   
                  2016-01-23-105         1         1            1     1   
                  2016-01-23-106         1         1            1     1   
                  2016-01-23-116         1         1            1     1   
                  2016-01-23-117         1         1            1     1   
                  2016-01-23-118         1         1            1     1   
                  2016-01-23-128         1         1            1     1   
                  2016-01-23-129         1         1            1     1   
                  2016-01-23-130         1         1            1     1   
                  2016-01-23-140         1         1            1     1   
                  2016-01-23-141         1         1            1     1   
                  2016-01-23-142         1         1            1     1   
                  2016-01-23-44          1         1            1     1   
                  2016-01-23-45          1         1            1     1   
                  2016-01-23-46          1         1            1     1   
                  2016-01-23-56          1         1            1     1   
                  2016-01-23-57          1         1            1     1   
                  2016-01-23-58          1         1            1     1   
                  2016-01-23-68          1         1            1     1   
                  2016-01-23-69          1         1            1     1   
                  2016-01-23-70          1         1            1     1   
                  2016-01-23-80          1         1            1     1   
                  2016-01-23-81          1         1            1     1   
                  2016-01-23-82          1         1            1     1   
                  2016-01-23-92          1         1            1     1   
                  2016-01-23-93          1         1            1     1   
                  2016-01-23-94          1         1            1     1   
                  2016-01-25-104         1         1            1     1   
                  2016-01-25-105         1         1            1     1   
...                                    ...       ...          ...   ...   
66                2016-01-29-92          1         1            1     1   
                  2016-01-29-93          1         1            1     1   
                  2016-01-29-94          1         1            1     1   
                  2016-01-31-104         1         1            1     1   
                  2016-01-31-105         1         1            1     1   
                  2016-01-31-106         1         1            1     1   
                  2016-01-31-116         1         1            1     1   
                  2016-01-31-117         1         1            1     1   
                  2016-01-31-118         1         1            1     1   
                  2016-01-31-128         1         1            1     1   
                  2016-01-31-129         1         1            1     1   
                  2016-01-31-130         1         1            1     1   
                  2016-01-31-140         1         1            1     1   
                  2016-01-31-141         1         1            1     1   
                  2016-01-31-142         1         1            1     1   
                  2016-01-31-44          1         1            1     1   
                  2016-01-31-45          1         1            1     1   
                  2016-01-31-46          1         1            1     1   
                  2016-01-31-56          1         1            1     1   
                  2016-01-31-57          1         1            1     1   
                  2016-01-31-58          1         1       

In [ ]:
dates=order_data.date
predictions=None
length=(np.arange(1,67,1))
length=pandas.DataFrame(length,columns=['val'],dtype=str)
predictions=pandas.DataFrame(length,columns=['val'],dtype=str)
predictions.val=predictions.val+'-'+dates[0]
predictions
for i,d in enumerate(dates):
    if i!=0:
        predictions=predictions.append(length.val+'-'+d,ignore_index=True)
#         print predictions
    if i%1000==0:
        print i
#         counter+=1
# predictions


In [65]:
uniqueset=[]
for i in xrange(66):
    uniqueset.extend(order_data[order_data.start_region_hash==4].date.unique())
    uniqueset=np.unique(uniqueset).tolist()

In [66]:
len(uniqueset)

133

In [113]:
def get_data_forprediction(meanedWeather,poi_dataframe,cluster_map):
    sample=pandas.read_table('./sample.csv',sep=',')
    sample.drop('gap',axis=1,inplace=True)
    sample['date']=sample.district_date_slot.apply(func)
    sample.district_date_slot=sample.district_date_slot.apply(returndistrict)
    sample.district_date_slot.replace(to_replace=cluster_map.index,value=cluster_map.region_id,inplace=True)

    sample=sample.join(poi_dataframe,on='district_date_slot',how='inner')
    sample=sample.join(meanedWeather,on='date',how='inner')
    col=sample.weather
    sample.drop('weather',axis=1,inplace=True)
    sample['weather']=col

    sample['week_day']=sample.date.apply(returnweekday)
    sample.date=sample.date.apply(returntimeslot)
    return sample

In [114]:
sample=get_data_forprediction(meanedWeather,poi_dataframe,cluster_map)
sample

,district_date_slot,date,poi_info,temperature,pm25,weather,week_day
0,1,1,5.219245,1.0,69.0,8.0,5
1,2,1,2.797114,1.0,69.0,8.0,5
2,3,1,0.882864,1.0,69.0,8.0,5
3,4,1,1.928195,1.0,69.0,8.0,5
4,5,1,0.453589,1.0,69.0,8.0,5
5,6,1,0.860071,1.0,69.0,8.0,5
6,7,1,9.843643,1.0,69.0,8.0,5
7,8,1,12.026212,1.0,69.0,8.0,5
8,9,1,1.936643,1.0,69.0,8.0,5
9,10,1,0.431584,1.0,69.0,8.0,5


In [25]:
sample.sort(['date','district_date_slot'])

/home/rameez/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,district_date_slot,date,poi_info,temperature,pm25,weather,week_day
0,1,1,5.219245,NaN,NaN,NaN,5
3366,1,1,5.219245,-4.0,56.0,2.0,0
6336,1,1,5.219245,-1.0,72.0,1.0,2
9702,1,1,5.219245,4.0,111.0,2.0,4
12672,1,1,5.219245,4.0,61.0,3.0,6
1,2,1,2.797114,NaN,NaN,NaN,5
3367,2,1,2.797114,-4.0,56.0,2.0,0
6337,2,1,2.797114,-1.0,72.0,1.0,2
9703,2,1,2.797114,4.0,111.0,2.0,4
12673,2,1,2.797114,4.0,61.0,3.0,6


In [115]:
sample.to_csv('predict_data',sep='\t')

In [ ]:
order_data.date=order_data.date.apply(datetimeslot)
order_data.start_region_hash.replace(to_replace=cluster_map.index,value=cluster_map.region_id,inplace=True)
# order_data.drop(labels=['order_id','passenger_id','dest_region_hash'],axis=1,inplace=True)

In [ ]:
grouped=order_data.groupby(['start_region_hash','date']).count()
grouped

In [ ]:
order_data=order_data.join(poi_dataframe,on='start_region_hash',how='inner')
order_data=order_data.join(meanedWeather,on='date')
order_data

In [ ]:
order_data['week_day']=order_data.date.apply(returnweekday)
order_data.date=order_data.date.apply(returntimeslot)

In [ ]:
order_data

In [ ]:
new=order_data[np.logical_not(order_data.weather.isnull())]

In [ ]:
new

# EXTRA THINGS

## Joining POI with Order and Writing

In [ ]:
os.chdir('./newdata')
chunk= order_data.shape[0]/21
string = 'file'
for i in xrange(21):
    if i==20:
        temp=order_data[i*chunk:].join(poi_dataframe,on='start_region_hash',how='inner').sort_index()
        temp.to_csv(string+str(i),sep='\t')
    else:
        temp=order_data[i*chunk:(i+1)*chunk].join(poi_dataframe,on='start_region_hash',how='inner').sort_index()
        temp.to_csv(string+str(i),sep='\t')
print 'done writing'
os.chdir('..')

## Reading with joined POI_DATA

In [ ]:
def read_with_poi(filenumbers,path):
    os.chdir('./'+path)
    string='file'
    order_data=pandas.read_table(string+str(filenumbers[0]),parse_dates=True, infer_datetime_format=True, keep_date_col=True,index_col=0)
    for i,f in enumerate(filenumbers):
        if i!=0:
            order_data=order_data.append(pandas.read_table(string+str(f),parse_dates=True, infer_datetime_format=True, keep_date_col=True,index_col=0))
    os.chdir('..')
    order_data.time=pandas.to_datetime(order_data.time).apply(correcttime)
    return order_data
# order_data=read_with_poi([0])

## Writing Final Data to files

In [ ]:
string = 'file'
for i in xrange(21):
    order_data=read_with_poi([i],'newdata')
    if i==20:
        temp=order_data.join(meanedWeather,on=['date','time_slots']).sort_index()       
        os.chdir('./final_data')
        temp.to_csv(string+str(i),sep='\t')
        os.chdir('..')
    else:
        temp=order_data.join(meanedWeather,on=['date','time_slots']).sort_index()
        os.chdir('./final_data')
        temp.to_csv(string+str(i),sep='\t')
        os.chdir('..')

## Update Final Dataset

In [ ]:
order_data=None

## Load Final Data Set

In [ ]:
d=read_with_poi([0],'final_data')

In [ ]:
# d.drop(labels=['order_id','driver_id','passenger_id','dest_region_hash'],axis=1,inplace=True)
# d=d.join(pandas.DataFrame(labels),on=['start_region_hash','date','time_slots'],how='inner')
# d


In [7]:
getTimeslot=lambda k : int(re.findall(r'.*?_(.*)',k)[0])
getDate=lambda k : re.findall(r'(.*?)_',k)[0]

In [8]:
os.chdir('..')
validation=pandas.read_csv('order_val_v2.csv')
os.chdir('./training_data')

In [9]:
order_data=pandas.read_table('final_data_complete',index_col=0)

In [10]:
validation['start_region_hash']=validation.id.apply(returndistrict)

In [11]:
validation['date']=validation.id.apply(func)

In [12]:
validation['time_slot']=validation.date.apply(getTimeslot)

In [13]:
validation['date']=validation.date.apply(getDate)

In [14]:
validation.drop('id',axis=1,inplace=True)

In [15]:
validation.start_region_hash=validation.start_region_hash.replace(cluster_map.index,cluster_map.region_id)

In [16]:
validation['day']=validation.date.apply(returnday)

In [17]:
validation.date=validation.time_slot
validation.drop('time_slot',axis=1,inplace=True)

In [18]:
typecast_int=lambda k : int(k)

In [19]:
validation.day=validation.day.apply(typecast_int)

In [21]:
validation.to_csv('val_match',sep='\t')

In [22]:
v=validation.set_index(['start_region_hash','day','date'])

In [23]:
validation.dtypes

gap                  int64
start_region_hash    int64
date                 int64
day                  int64
dtype: object

In [26]:
order_data['currentgap']=labels.gap

In [27]:
order_data=order_data.join(v,on=['start_region_hash','day','date'])

In [28]:
order_data.gap=order_data.gap.fillna(-1)

In [32]:
check=order_data[order_data.gap>-1]

In [34]:
check.drop('currentgap',axis=1,inplace=True)

/home/rameez/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [35]:
check

,start_region_hash,date,number_of_orders,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8#3,8#4,8#5,9#1#1,temperature,pm25,weather,week_day,day,gap
2592,1,0,77,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.000000,195.000000,2.0,1,19,5.0
2593,1,1,51,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.000000,195.000000,2.0,1,19,3.0
2594,1,10,30,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,2.000000,199.000000,1.0,1,19,1.0
2595,1,100,208,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,12.0
2596,1,101,165,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,1.0
2597,1,102,256,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,4.0
2598,1,103,216,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,6.0
2599,1,104,222,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.600000,94.000000,2.0,1,19,9.0
2600,1,105,249,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.000000,91.000000,2.0,1,19,1.0
2601,1,106,251,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.000000,91.000000,2.0,1,19,5.0


In [33]:
check.to_csv('validation_set',sep='\t')

,start_region_hash,date,number_of_orders,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8#4,8#5,9#1#1,temperature,pm25,weather,week_day,day,currentgap,gap
2592,1,0,77,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,3.000000,195.000000,2.0,1,19,5,5.0
2593,1,1,51,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,3.000000,195.000000,2.0,1,19,3,3.0
2594,1,10,30,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,2.000000,199.000000,1.0,1,19,1,1.0
2595,1,100,208,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,12,12.0
2596,1,101,165,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,1,1.0
2597,1,102,256,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,4,4.0
2598,1,103,216,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,4.000000,96.000000,2.0,1,19,6,6.0
2599,1,104,222,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,3.600000,94.000000,2.0,1,19,9,9.0
2600,1,105,249,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,3.000000,91.000000,2.0,1,19,1,1.0
2601,1,106,251,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,2075.0,0.0,0.0,3.000000,91.000000,2.0,1,19,5,5.0


In [73]:
order_data

,start_region_hash,date,number_of_orders,1,1#1,1#10,1#11,1#2,1#3,1#4,...,8#3,8#4,8#5,9#1#1,temperature,pm25,weather,week_day,day,currentgap
0,1,0,187,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.500000,177.000000,1.0,4,1,9
1,1,1,198,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.000000,177.000000,1.0,4,1,7
2,1,10,72,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,3.000000,206.000000,1.0,4,1,2
3,1,100,279,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,12.000000,168.000000,2.0,4,1,5
4,1,101,238,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,11.000000,154.000000,2.0,4,1,3
5,1,102,283,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,11.000000,154.000000,2.0,4,1,9
6,1,103,275,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,11.000000,154.000000,2.0,4,1,7
7,1,104,280,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,10.000000,154.000000,2.0,4,1,7
8,1,105,290,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,10.000000,154.000000,2.0,4,1,11
9,1,106,293,2739.0,498.0,498.0,3984.0,1909.0,0.0,83.0,...,1743.0,2075.0,0.0,0.0,10.000000,146.000000,2.0,4,1,8


In [ ]:
validation